In [ ]:
# Install system dependencies
!apt-get update && apt-get install -y libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1 libgthread-2.0-0

In [ ]:
# Install Python packages
!pip install torch==2.8.0 torchvision==0.23.0 --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics==8.3.208 onnxruntime==1.22.0 onnx==1.16.1
!pip install opencv-python==4.12.0.88 numpy==1.26.4 pillow==11.3.0 tqdm==4.67.1
!pip install simple-lama-inpainting sympy==1.14.0
!pip install matplotlib seaborn pandas ipywidgets

In [ ]:
!wget -O yolov8n-seg.pt https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n-seg.pt

In [ ]:
import torch
import onnxruntime as ort
import numpy as np
import cv2

print('=' * 50)
print('Google Colab Environment Verification')
print('=' * 50)
print(f'PyTorch: {torch.__version__}')
print(f'CUDA Available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA Version: {torch.version.cuda}')
    print(f'GPU Device: {torch.cuda.get_device_name(0)}')
print(f'NumPy: {np.__version__}')
print(f'OpenCV: {cv2.__version__}')
print(f'ONNX Runtime: {ort.__version__}')
print(f'Available Providers: {ort.get_available_providers()}')
print('=' * 50)
print('✅ Colab setup complete!')
print('=' * 50)

In [ ]:

# Test imports and basic functionality
from ultralytics import YOLO
import torch
from simple_lama_inpainting import SimpleLama
import onnxruntime as ort

print("Testing YOLO...")
yolo_model = YOLO('yolov8n-seg.pt')
print("✅ YOLO loaded")

print("Testing LaMa...")
# Note: LaMa model would need to be available
# lama_model = SimpleLama()
print("✅ LaMa ready")

print("Testing ONNX Runtime...")
print(f"Available providers: {ort.get_available_providers()}")
print("✅ ONNX Runtime ready")

print("🎉 All components ready!")


All Above steps given from Jeffrey
Everything below is Matt's rendition on the mvp


In [ ]:
This is Frame by Frame using Yolo 11x segmentation Model + Lama Inpainting Model

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
import onnxruntime as ort
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow

# Initialize models
yolo_model = YOLO('yolov11x-seg.pt')

# Simple LaMa inpainting model
# The LaMa model needs to be downloaded or provided.
# For this example, we will use a placeholder and assume it's available.
# In a real scenario, you would initialize SimpleLama with the path to your model weights.
try:
    lama_model = SimpleLama()
except Exception as e:
    print(f"Could not initialize SimpleLama. Please ensure the model is available. Error: {e}")
    # As a workaround for demonstration, we'll create a dummy inpainting function
    # In a real application, replace this with the actual LaMa inpainting logic
    def inpaint_frame(frame, mask):
        # Simple placeholder: blur the masked area
        if mask is not None and np.sum(mask) > 0:
            mask_indices = np.where(mask > 0)
            # Simple inpainting - replace masked area with a solid color (e.g., black)
            frame[mask_indices] = [0, 0, 0]
        return frame
    print("Using a dummy inpainting function. Please replace with actual LaMa implementation.")
    lama_inpaint = inpaint_frame

# Load Video from File
video_path = '/content/Test Video.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

# VIdeo Properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Saving Output after
output_video_path = 'output_video.mp4'
# fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("Processing video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # YOLO Object Detection
    results = yolo_model(frame)

    # Mask from segmentation results (looking at cars)
    mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
    if results and results[0].masks:
        for i, segment in enumerate(results[0].masks.xy):
            # Check if the detected object is a 'car' (class ID 2 in COCO dataset)
            if results[0].boxes.cls[i] == 2:
                segment_int = np.array(segment, dtype=np.int32)
                cv2.fillPoly(mask, [segment_int], 255)


    # Apply inpainting using Simple LaMa
    # Ensure the LaMa model is properly initialized and used here
    # For the dummy function, we use it directly
    if 'lama_model' in locals():
        try:
            # Convert OpenCV image (BGR, numpy array) to PIL Image (RGB)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)
            mask_pil = Image.fromarray(mask)

            # Perform inpainting
            inpainted_img_pil = lama_model(img_pil, mask_pil)

            # Convert PIL Image back to OpenCV format (BGR, numpy array)
            inpainted_frame = cv2.cvtColor(np.array(inpainted_img_pil), cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"Error during LaMa inpainting: {e}")
            inpainted_frame = frame # Fallback to original frame on error
    else:
        # Use the dummy inpainting function if LaMa model is not initialized
        inpainted_frame = lama_inpaint(frame.copy(), mask)


    # Display the processed frame in Colab
    cv2_imshow(inpainted_frame)

# Release resources
cap.release()
cv2.destroyAllWindows() # This might not be necessary/effective with cv2_imshow

print("Video processing finished.")